# Hacking Dask clusters

This notebook covers Dask's distributed clusters in more detail. We provide a more in depth look at the components of a cluster, illustrate how to inspect the internal state of a cluster, and how you can extend the functionality of your cluster using Dask's plugin system.

# Cluster overview

In this section we'll discuss:

1. The different components which make up a Dask cluster
2. Survey different ways to launch a cluster

## Components of a cluster

A Dask cluster is composed of three different types of objects:

1. **Scheduler**: A single, centralized scheduler process which responds to requests for computations, maintains relavant state about tasks and worker, and sends tasks to workers to be computed.
2. **Workers**: One or more worker processes which compute tasks and store/serve their results.
3. **Clients**: One or more client objects which are the user-facing entry point to interact with the cluster.

<img src="images/dask-cluster.png"
     width="90%"
     alt="Dask components\">

A couple of notes about workers:

- Each worker runs in its own Python process. Each worker Python process has its own `concurrent.futures.ThreadPoolExecutor` which is uses to compute tasks in parallel. The same threads vs. processes considerations we discussed earlier also apply to Dask workers.
- There's actually a fourth cluster object which is often not discussed: the **Nanny**. By default Dask workers are launched and managed by a separate nanny process. This separate process allows workers to restart themselves if you want to use the `Client.restart` method, or to restart workers automatically if they get above a certain memory limit threshold.

#### Related Documentation

- [Cluster architecture](https://distributed.dask.org/en/latest/#architecture)
- [Journey of a task](https://distributed.dask.org/en/latest/journey.html)

## Deploying Dask clusters

Deploying a Dask cluster means launching scheduler, worker, and client processes and setting up the appropriate network connections so these processes can communicate with one another. Dask clusters can be lauched in a few different ways which we highlight in the following sections.

### Manual setup

Launch a scheduler process using the `dask-scheduler` command line utility:

```terminal
$ dask-scheduler
Scheduler at:   tcp://192.0.0.100:8786
```

and then launch several workers by using the `dask-worker` command and providing them the address of the scheduler they should connect to:

```terminal
$ dask-worker tcp://192.0.0.100:8786
Start worker at:  tcp://192.0.0.1:12345
Registered to:    tcp://192.0.0.100:8786

$ dask-worker tcp://192.0.0.100:8786
Start worker at:  tcp://192.0.0.2:40483
Registered to:    tcp://192.0.0.100:8786

$ dask-worker tcp://192.0.0.100:8786
Start worker at:  tcp://192.0.0.3:27372
Registered to:    tcp://192.0.0.100:8786
```

### Python API (advanced)

⚠️ **Warning**: Creating `Scheduler` / `Worker` objects explicitly in Python is rarely needed in practice and is intended for more advanced users ⚠️

In [ ]:
from dask.distributed import Scheduler, Worker, Client

# Launch a scheduler
async with Scheduler() as scheduler: # Launch a scheduler
    # Launch a worker which connects to the scheduler
    async with Worker(scheduler.address) as worker:
        # Launch a client which connects to the scheduler
        async with Client(scheduler.address, asynchronous=True) as client:
            result = await client.submit(sum, range(100))
            print(f"{result = }")

### Cluster managers (recommended)

Dask has the notion of cluster manager objects. Cluster managers offer a consistent interface for common activities like adding/removing workers to a cluster, retrieving logs, etc.

In [ ]:
from dask.distributed import LocalCluster

# Launch a scheduler and 4 workers on my local machine
cluster = LocalCluster(n_workers=4, threads_per_worker=2)
cluster

In [ ]:
# Scale up to 10 workers
cluster.scale(10)

In [ ]:
# Scale down to 2 workers
cluster.scale(2)

In [ ]:
# Retrieve cluster logs
cluster.get_logs()

In [ ]:
# Shut down cluster
cluster.close()

There are several projects in the Dask ecosystem for easily deploying clusters on commonly used computing resources:

- [Dask-Kubernetes](https://kubernetes.dask.org/en/latest/) for deploying Dask using native Kubernetes APIs
- [Dask-Cloudprovider](https://cloudprovider.dask.org/en/latest/) for deploying Dask clusters on various cloud platforms (e.g. AWS, GCP, Azure, etc.)
- [Dask-Yarn](https://yarn.dask.org/en/latest/) for deploying Dask on YARN clusters
- [Dask-MPI](http://mpi.dask.org/en/latest/) for deploying Dask on existing MPI environments
- [Dask-Jobqueue](https://jobqueue.dask.org/en/latest/) for deploying Dask on job queuing systems (e.g. PBS, Slurm, etc.)

Launching clusters with any of these projects follows a similar pattern as using Dask's built-in `LocalCluster`:

```python
# Launch a Dask cluster on a Kubernetes cluster
from dask_kubernetes import KubeCluster
cluster = KubeCluster(...)

# Launch a Dask cluster on AWS Fargate
from dask_cloudprovider.aws import FargateCluster
cluster = FargateCluster(...)

# Launch a Dask cluster on a PBS job queueing system
from dask_jobqueue import PBSCluster
cluster = PBSCluster(...)
```

Additionally, there are compaines like [Coiled](https://coiled.io) and [Saturn Cloud](https://www.saturncloud.io) which have Dask deployment-as-a-service offerings. *Disclaimer*: the instructors for this tutorial are employed both by these comapnies. 

#### Related Documentation

- [Cluster setup](https://docs.dask.org/en/latest/setup.html)

# Inspecting a cluster's state

In this section we'll:

1. Familiarize ourselves with Dask's scheduler and worker processes
2. Explore the various state that's tracked throughout the cluster
3. Learn how to inspect remote scheduler and worker processes

Dask has a a variety of ways to provide users insight into what's going on during their computations. For example, Dask's [diagnositc dashboard](https://docs.dask.org/en/latest/diagnostics-distributed.html) displays real-time information about what tasks are current running, overal progress on a computation, worker CPU and memory load, statistical profiling information, and much more. Additionally, Dask's [performance reports](https://distributed.dask.org/en/latest/diagnosing-performance.html#performance-reports) allow you to save the diagnostic dashboards as static HTML plots. Performance reports are particularly useful when benchmarking/profiling workloads or when sharing workload performance with colleagues.

In [ ]:
from dask.distributed import LocalCluster, Client, Worker

cluster = LocalCluster(worker_class=Worker)
client = Client(cluster)
client

In [ ]:
import dask.array as da
from dask.distributed import performance_report

with performance_report("my_report.html"):
    x = da.random.random((10_000, 10_000), chunks=(1_000, 1_000))
    result = (x + x.T).mean(axis=0).mean()
    result.compute()

These are invaluable tools and we highly recommend utilizing them. Often times Dask's dashboard is totally sufficient to understand the performance of your computations.

However, sometimes it can be useful to dive more deeply into the internals of your cluster and directly inspect the state of your scheduler and workers. Let's start by submitting some tasks to the cluster to be computed.

In [ ]:
import random

def double(x):
    random.seed(x)
    # Simulate some random task failures
    if random.random() < 0.1:
        raise ValueError("Oh no!")
    return 2 * x

futures = client.map(double, range(50))

One of the nice things about `LocalCluster` is it gives us direct access the `Scheduler` Python object. This allows us to easily inspect the scheduler directly.

In [ ]:
scheduler = cluster.scheduler
scheduler

ℹ️ Note that often times you won't have direct access to the `Scheduler` Python object (e.g. when the scheduler is running on separate machine). In these cases it's still possible to inspect the scheduler and we will discuss how to do this later on.

The scheduler tracks **a lot** of state. Let's start to explore the scheduler to get a sense for what information it keeps track of.

In [ ]:
scheduler.address   # Scheduler's address

In [ ]:
scheduler.time_started   # Time the scheduler was started

In [ ]:
dict(scheduler.workers)

In [ ]:
worker_state = next(iter(scheduler.workers.values()))
worker_state

Let's take a look at the `WorkerState` attributes

In [ ]:
worker_state.address   # Worker's address

In [ ]:
worker_state.status   # Current status of the worker (e.g. "running", "closed")

In [ ]:
worker_state.nthreads   # Number of threads in the worker's `ThreadPoolExecutor`

In [ ]:
worker_state.executing   # Dictionary of all tasks which are currently being processed, along with the current duration of the task

In [ ]:
worker_state.metrics   # Various metrics describing the current state of the worker

Workers check in with the scheduler inform it when certain event occur (e.g. when a worker has completed a task) so the scheduler can update it's internal state.

In [ ]:
worker_state.last_seen

In [ ]:
import time

for _ in range(10):
    print(f"{worker_state.last_seen = }")
    time.sleep(1)

In addition to the state of each worker, the scheduler also tracks information for each task it has been asked to run.

In [ ]:
scheduler.tasks

In [ ]:
task_state = next(iter(scheduler.tasks.values()))

In [ ]:
task_state

In [ ]:
task_state.key   # Task's name (unique identifier)

In [ ]:
task_state.state   # Task's state (e.g. "memory", "waiting", "processing", "erred", etc.)

In [ ]:
task_state.who_has   # Set of workers (`WorkerState`s) who have this task's result in memory

In [ ]:
task_state.nbytes   # The number of bytes of the result of this finished task

In [ ]:
task_state.type   # The type of the the task's result (as a string)

In [ ]:
task_state.retries   # The number of times this task can automatically be retried in case of failure

## Exercise 1

Spend the next 5 minutes continuing to explore the attributes the scheduler keeps track of. Try to answer the following questions:

1. What are the keys for the tasks which failed?
2. How many tasks successfully ran on each worker?

In [ ]:
# What are the keys for the tasks which failed?
# Your solution goes here

In [ ]:
# Solution to "What are the keys for the tasks which failed?"
erred_tasks = [key for key, ts in scheduler.tasks.items() if ts.state == "erred"]
erred_tasks

In [ ]:
# How many tasks successfull ran on each worker?
# Your solution goes here

In [ ]:
# Solution to "How many tasks successfull ran on each worker?"
from collections import defaultdict

erred_tasks = [key for key, ts in scheduler.tasks.items() if ts.state == "erred"]
counter = defaultdict(int)
for key in scheduler.tasks:
    if key in erred_tasks:
        continue
    for worker in scheduler.who_has[key]:
        counter[worker] += 1
print(counter)

# # Alternative solution to "How many tasks successfull ran on each worker?"
# counter = {address: worker_state.metrics["in_memory"]
#            for address, worker_state in scheduler.workers.items()}
# print(counter)

In addition to inspecting the scheduler, we can also investigate the state of each of our workers.

In [ ]:
cluster.workers

In [ ]:
worker = next(iter(cluster.workers.values()))
worker

In [ ]:
worker.address   # Worker's address

In [ ]:
worker.executing_count   # Number of tasks the worker is currenting computing

In [ ]:
worker.executed_count   # Running total of all tasks processed on this worker

In [ ]:
worker.nthreads   # Number of threads in the worker's ThreadPoolExecutor

In [ ]:
worker.executor   # Worker's ThreadPoolExecutor where it computes tasks

In [ ]:
worker.keys()   # Keys the worker currently has in memory

In [ ]:
worker.data   # Where the worker stores task results

In [ ]:
{key: worker.data[key] for key in worker.keys()}

## Accessing remote scheduler and workers

As we noted earlier, often times you won't have direct access to the `Scheduler` or `Worker` Python objects for your cluster. However, in these cases it's still possible to examine the state of the scheduler and workers in your cluster using the `Client.run` ([docs](https://distributed.dask.org/en/latest/api.html#distributed.Client.run)) and `Client.run_on_scheduler`([docs](https://distributed.dask.org/en/latest/api.html#distributed.Client.run_on_scheduler)) methods.

`Client.run` allows you to run a function on worker processes in your cluster. If the function has a `dask_worker` parameter, then that variable will be populated with the `Worker` instance when the function is run. Likewise, `Client.run_on_scheduler` allows you to run a function on the scheduler processes in your cluster. If the function has a `dask_scheduler` parameter, then that variable will be populated with the `Scheduler` instance when the function is run.

Let's look at some examples.

In [ ]:
import os

result = client.run(os.getpid)
result

`Client.run` also accepts a `workers=` keyword argument which is the list of workers you want to run the specified function on (by default it will run on all workers in the cluster).

In [ ]:
workers = list(result.keys())[:2]
workers

In [ ]:
import os

client.run(os.getpid, workers=workers)

You can even run custom function you've written yourself! If the function has a `dask_worker` parameter ...

In [ ]:
def get_worker_name(dask_worker):
    return dask_worker.name

client.run(get_worker_name)

Similarly, we can do the same thing on the scheduler by using `Client.run_on_scheduler`

In [ ]:
client.run_on_scheduler(os.getpid)

In [ ]:
def get_erred_tasks(dask_scheduler):
    return [key for key, ts in dask_scheduler.tasks.items() if ts.state == "erred"]

client.run_on_scheduler(get_erred_tasks)

In [ ]:
client.close()
cluster.close()

#### Related Documentation

- [Dask worker](https://distributed.dask.org/en/latest/worker.html)
- [Scheduling state](https://distributed.dask.org/en/latest/scheduling-state.html)

# Extending the scheduler and workers: Dask's plugin system

In this section we'll siscuss Dask's scheduler and worker plugin systems and write our own plugin to extend the scheduler's functionality.

So far we've primarily focused on inspecting the state of a cluster. However, there are times when it's useful to extend the functionality of the scheduler and/or workers in a cluster. To help facilitate this, Dask has scheduler and worker plugin systems which enable you to hook into different events that happen throughout a cluster's lifecycle. This allows you to run custom code when a specific type of event occurs on the cluster.

Specifically, the [scheduler plugin system](https://distributed.dask.org/en/latest/plugins.html#scheduler-plugins) enables you run custom code when the following events occur:

1. Scheduler starts, stops, or is restarted
2. Client connects or disconnects to the scheduler
3. Workers enters or leaves the cluster
4. When a new task enters the scheduler
5. When a task changes state (e.g. from "processing" to "memory")

While the [worker plugin system](https://distributed.dask.org/en/latest/plugins.html#worker-plugins) enables you run custom code when the following events occur:

1. Worker starts or stops
2. When a worker releases a task
3. When a task changes state (e.g. "processing" to "memory")

Implementing your own custom plugin consists of creating a Python class with certain methods (each method corresponds to a particular lifecycle event).

In [ ]:
from distributed import SchedulerPlugin, WorkerPlugin

In [ ]:
# Lifecycle SchedulerPlugin methods
[attr for attr in dir(SchedulerPlugin) if not attr.startswith("_")]

In [ ]:
# Lifecycle WorkerPlugin methods
[attr for attr in dir(WorkerPlugin) if not attr.startswith("_")]

For the exact signature of each method, please refer to the [`SchedulerPlugin`](https://distributed.dask.org/en/latest/plugins.html#scheduler-plugins) and [`WorkerPlugin`](https://distributed.dask.org/en/latest/plugins.html#worker-plugins) documentation.

Let's looks at an example scheduler plugin.

In [ ]:
class Counter(SchedulerPlugin):
    """Keeps a running count of the total number of completed tasks"""
    def __init__(self):
        self.n_tasks = 0

    def transition(self, key, start, finish, *args, **kwargs):
        if start == "processing" and finish == "memory":
            self.n_tasks += 1

    def restart(self, scheduler):
        self.n_tasks = 0

To add a custom scheduler plugin to your cluster, use the `Scheduler.add_plugin` method:

In [ ]:
# Create LocalCluster and Client
cluster = LocalCluster()
client = Client(cluster)

# Instantiate and add the Counter to our cluster
counter = Counter()
cluster.scheduler.add_plugin(counter)

In [ ]:
counter.n_tasks

In [ ]:
from distributed import wait
futures = client.map(lambda x: x + 1, range(27))
wait(futures);

In [ ]:
counter.n_tasks

In [ ]:
client.close()
cluster.close()

This is a relatively straightforward plugin one could write. Let's look at the `distributed`s built-in `PipInstall` worker plugin to see a more real-world example.

In [ ]:
from distributed import PipInstall

PipInstall??

To add a custom worker plugin to your cluster, use the `Client.register_worker_plugin` method.

## Exercise 2

Over the next 10 minutes, create a `TaskTimerPlugin` scheduler plugin which keeps tracks of how long each task takes to run.

```python

class TaskTimerPlugin(SchedulerPlugin):
    ...

# Create LocalCluster and Client
cluster = LocalCluster()
client = Client(cluster)

# Instantiate and add the TaskTimerPlugin to our cluster
plugin = TaskTimerPlugin()
cluster.scheduler.add_plugin(plugin)

import dask.array as da

x = da.random.random((20_000, 20_000), chunks=(5_000, 1_000))
result = (x + x.T).mean(axis=0).sum()
result.compute()
```

In [ ]:
# Your solution to Exercise 2 here

In [ ]:
# Solution to Exercise 2
import time

class TaskTimerPlugin(SchedulerPlugin):
    def __init__(self):
        self.start_times = {}
        self.stop_times = {}
        self.task_durations = {}

    def transition(self, key, start, finish, *args, **kwargs):
        if finish == "processing":
            self.start_times[key] = time.time()
        elif finish == "memory":
            self.stop_times[key] = time.time()
            self.task_durations[key] = self.stop_times[key] - self.start_times[key]

# Create LocalCluster and Client
cluster = LocalCluster()
client = Client(cluster)

# Instantiate and add the TaskTimerPlugin to our cluster
plugin = TaskTimerPlugin()
cluster.scheduler.add_plugin(plugin)

import dask.array as da

x = da.random.random((20_000, 20_000), chunks=(5_000, 1_000))
result = (x + x.T).mean(axis=0).sum()
result.compute()

plugin.task_durations

**Bonus**: If you have extra time, make a plot of the task duration distribution (hint: `pandas` and `matplotlib` are installed)

In [ ]:
# Your plotting code here

In [ ]:
import pandas as pd

df = pd.DataFrame([(key, 1_000 * value) for key, value in plugin.task_durations.items()],
                  columns=["key", "duration"])
ax = df.duration.plot(kind="hist", bins=50, logy=True)
ax.set_xlabel("Task duration [ms]")
ax.set_ylabel("Counts");

In [ ]:
client.close()
cluster.close()

# Summary

This notebook we took a detailed look at the components of a Dask cluster, illustrated how to inspect the internal state of a cluster (both the scheduler and workers), and how you can use Dask's plugin system to execute custom code during a cluster's lifecycle.

# Additional Resources

- Repositories on GitHub:
    - Dask https://github.com/dask/dask
    - Distributed https://github.com/dask/distributed
    
- Documentation:
    - Dask documentation https://docs.dask.org
    - Distributed documentation https://distributed.dask.org

- If you have a Dask usage questions, please ask it on the [Dask GitHub discussions board](https://github.com/dask/dask/discussions).

- If you run into a bug, feel free to file a report on the [Dask GitHub issue tracker](https://github.com/dask/dask/issues).

- If you're interested in getting involved and contributing to Dask. Please check out our [contributing guide](https://docs.dask.org/en/latest/develop.html).

# Thank you!